## Interpreting the evaluation results of 0208

The first benchmark

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
# Read the log
df = pd.read_csv('../../models/output/eval-log.csv')
df['query'] = df['code_sample_idx'].apply(str) + '. ' + df['query']

In [62]:
from __future__ import division
df['prank'] = (df['rank'] - 1) / df['num_alt']

In [60]:
df.shape

(8305, 10)

In [63]:
df.head()

,code_sample_idx,query,func,baseline,candidate,rank,is_gt,score,num_alt,prank
0,1,1. change the texture patterns of the bar,NaN,BM25,bar,1,0,10.633178,9,0.000000
1,1,1. change the texture patterns of the bar,NaN,BM25,set_hatch,2,1,7.838210,9,0.111111
2,1,1. change the texture patterns of the bar,NaN,BM25,get_cmap,3,0,0.000000,9,0.222222
3,1,1. change the texture patterns of the bar,NaN,BM25,set_width,4,0,0.000000,9,0.333333
4,1,1. change the texture patterns of the bar,NaN,BM25,set_label,5,0,0.000000,9,0.444444


### Question 1: Are all the systems failing on the same questions?

### func

In [64]:
df_func_gt = df[df.func.isnull() & df.is_gt == 1]

In [65]:
df_func_gt_pivot = df_func_gt[['query','baseline', 'rank']].pivot_table(
    index='query',columns='baseline',aggfunc=min)

In [66]:
df_func_gt_norm_pivot = df_func_gt[['query','baseline', 'prank']].pivot_table(
    index='query',columns='baseline',aggfunc=min)

### arg

In [81]:
df_arg_gt = df[~df.func.isnull() & df.is_gt == 1]

df_arg_gt_pivot = df_arg_gt[['query','func','baseline', 'rank']].pivot_table(
    index=['query','func'],columns='baseline',aggfunc=min)

df_arg_gt_norm_pivot = df_arg_gt[['query','func','baseline', 'prank']].pivot_table(
    index=['query','func'],columns='baseline',aggfunc=min)

In [84]:
writer = pd.ExcelWriter('funcarg_gt_pivot.xls')
df_func_gt_pivot.to_excel(writer, 'func')
df_func_gt_norm_pivot.to_excel(writer, 'func_norm')
df_arg_gt_pivot.to_excel(writer, 'arg')
df_arg_gt_norm_pivot.to_excel(writer, 'arg_norm')
writer.save()

## Dig Deeper: Why n-gram word2vec is performing worse??

### query: "1. set the color of the bar"

In [91]:
df[df.func.isnull() & (df.baseline=='BM25') & (df['query'] == '1. set the color of the bar')]

,code_sample_idx,query,func,baseline,candidate,rank,is_gt,score,num_alt,prank
924,1,1. set the color of the bar,NaN,BM25,bar,1,1,10.633178,9,0.000000
925,1,1. set the color of the bar,NaN,BM25,set_width,2,0,4.178821,9,0.111111
926,1,1. set the color of the bar,NaN,BM25,set_hatch,3,0,4.178821,9,0.222222
927,1,1. set the color of the bar,NaN,BM25,set_label,4,0,3.973527,9,0.333333
928,1,1. set the color of the bar,NaN,BM25,title,5,0,3.973527,9,0.444444
929,1,1. set the color of the bar,NaN,BM25,xlim,6,0,3.793140,9,0.555556
930,1,1. set the color of the bar,NaN,BM25,xticks,7,0,3.490922,9,0.666667
931,1,1. set the color of the bar,NaN,BM25,get_cmap,8,0,0.000000,9,0.777778
932,1,1. set the color of the bar,NaN,BM25,legend,9,0,0.000000,9,0.888889


In [92]:
df[df.func.isnull() 
   & (df.baseline=='vectors-so-text-python-5gram.bin_func_as_seq') 
   & (df['query'] == '1. set the color of the bar')]

,code_sample_idx,query,func,baseline,candidate,rank,is_gt,score,num_alt,prank
987,1,1. set the color of the bar,NaN,vectors-so-text-python-5gram.bin_func_as_seq,xlim,1,0,0.787440,9,0.000000
988,1,1. set the color of the bar,NaN,vectors-so-text-python-5gram.bin_func_as_seq,xticks,2,0,0.735804,9,0.111111
989,1,1. set the color of the bar,NaN,vectors-so-text-python-5gram.bin_func_as_seq,title,3,0,0.729406,9,0.222222
990,1,1. set the color of the bar,NaN,vectors-so-text-python-5gram.bin_func_as_seq,set_hatch,4,0,0.694642,9,0.333333
991,1,1. set the color of the bar,NaN,vectors-so-text-python-5gram.bin_func_as_seq,set_width,5,0,0.691684,9,0.444444
992,1,1. set the color of the bar,NaN,vectors-so-text-python-5gram.bin_func_as_seq,set_label,6,0,0.642508,9,0.555556
993,1,1. set the color of the bar,NaN,vectors-so-text-python-5gram.bin_func_as_seq,get_cmap,7,0,0.638531,9,0.666667
994,1,1. set the color of the bar,NaN,vectors-so-text-python-5gram.bin_func_as_seq,legend,8,0,0.549457,9,0.777778
995,1,1. set the color of the bar,NaN,vectors-so-text-python-5gram.bin_func_as_seq,bar,9,1,0.542181,9,0.888889


In [123]:
funcs = \
df[df.func.isnull() 
   & (df.baseline=='vectors-so-text-python-5gram.bin_func_as_seq') 
   & (df['query'] == '1. set the color of the bar')].candidate.tolist()
funcs

['xlim',
 'xticks',
 'title',
 'set_hatch',
 'set_width',
 'set_label',
 'get_cmap',
 'legend',
 'bar']

In [102]:
%cd ../../models

/Users/ronxin/Dropbox/git/codemend/models


In [104]:
import sys
sys.path.append('../models/')
from annotate_code_with_api import get_fu_fau

In [105]:
fu, fau = get_fu_fau()

In [107]:
fu['xlim']

'get or set the x limits of the current axes .'

In [108]:
fu['bar']

'make a bar plot .'

In [109]:
from word2vec_baseline import Word2vecBaseline

In [112]:
w2v = Word2vecBaseline(
                 'output/vectors-so-text-python-5gram.bin',
                 maxngram=5,
                 fu_fau=(fu,fau))

In [113]:
[w2v.model.index2word[x] for x in w2v.get_word_indexes(fu['bar'] + ' bar')]

['make', 'a', 'bar_plot', '.', 'bar']

In [114]:
[w2v.model.index2word[x] for x in w2v.get_word_indexes(fu['xlim'] + ' xlim')]

['get',
 'or',
 'set',
 'the',
 'x_limits',
 'of',
 'the',
 'current_axes',
 '.',
 'xlim']

In [115]:
[w2v.model.index2word[x] for x in w2v.get_word_indexes('set the color of the bar')]

['set', 'the', 'color', 'of', 'the', 'bar']

In [116]:
w2v.get_word_indexes('set the color of the bar')

[75, 1, 911, 15, 1, 2077]

In [118]:
np.dot(w2v.get_bow_representation(fu['bar'] + ' bar'),
       w2v.get_bow_representation('set the color of the bar').T)

0.49544877

In [119]:
np.dot(w2v.f_vecmat[w2v.func_lookup['bar']],
       w2v.get_bow_representation('set the color of the bar').T)

0.49544877

In [120]:
np.dot(w2v.f_vecmat[w2v.func_lookup['xlim']],
       w2v.get_bow_representation('set the color of the bar').T)

0.80199325

In [124]:
w2v.rank_funcs('set the color of the bar', funcs)

[('xlim', 0.80199326185821751),
 ('xticks', 0.75373183641630837),
 ('title', 0.71177487726350697),
 ('set_hatch', 0.69464193441954392),
 ('set_width', 0.69168423410661128),
 ('set_label', 0.64250774355492346),
 ('get_cmap', 0.63853126243112146),
 ('legend', 0.54945666001272375),
 ('bar', 0.49544878649104351)]

In [125]:
w2v1g = Word2vecBaseline(
                 w2v.model,
                 maxngram=1,
                 fu_fau=(fu,fau))

In [126]:
w2v1g.rank_funcs('set the color of the bar', funcs)

[('xlim', 0.78744002689626891),
 ('xticks', 0.7358038256871624),
 ('title', 0.7294064693709218),
 ('set_hatch', 0.69464193441954392),
 ('set_width', 0.69168423410661128),
 ('set_label', 0.64250774355492346),
 ('get_cmap', 0.63853126243112146),
 ('legend', 0.54945666001272375),
 ('bar', 0.54218060892628928)]

### query: "2. add label to the slices"

In [128]:
df[df.func.isnull() 
   & (df.baseline=='BM25') 
   & (df['query'] == '2. add label to the slices')]

,code_sample_idx,query,func,baseline,candidate,rank,is_gt,score,num_alt,prank
1606,2,2. add label to the slices,NaN,BM25,pie,1,1,0,4,0.00
1607,2,2. add label to the slices,NaN,BM25,title,2,0,0,4,0.25
1608,2,2. add label to the slices,NaN,BM25,set_bbox,3,0,0,4,0.50
1609,2,2. add label to the slices,NaN,BM25,axis,4,0,0,4,0.75


In [127]:
df[df.func.isnull() 
   & (df.baseline=='vectors-so-text-python-5gram.bin_func_as_seq') 
   & (df['query'] == '2. add label to the slices')]

,code_sample_idx,query,func,baseline,candidate,rank,is_gt,score,num_alt,prank
1634,2,2. add label to the slices,NaN,vectors-so-text-python-5gram.bin_func_as_seq,axis,1,0,0.563804,4,0.00
1635,2,2. add label to the slices,NaN,vectors-so-text-python-5gram.bin_func_as_seq,title,2,0,0.545443,4,0.25
1636,2,2. add label to the slices,NaN,vectors-so-text-python-5gram.bin_func_as_seq,set_bbox,3,0,0.367135,4,0.50
1637,2,2. add label to the slices,NaN,vectors-so-text-python-5gram.bin_func_as_seq,pie,4,1,0.358241,4,0.75


### query: "4. change the line type of the grid lines"

In [129]:
df[df.func.isnull() 
   & (df.baseline=='BM25') 
   & (df['query'] == '4. change the line type of the grid lines')]

,code_sample_idx,query,func,baseline,candidate,rank,is_gt,score,num_alt,prank
4497,4,4. change the line type of the grid lines,NaN,BM25,grid,1,1,16.669619,26,0.000000
4498,4,4. change the line type of the grid lines,NaN,BM25,plot,2,0,13.021258,26,0.038462
4499,4,4. change the line type of the grid lines,NaN,BM25,tick_params,3,0,7.444959,26,0.076923
4500,4,4. change the line type of the grid lines,NaN,BM25,figure,4,0,0.000000,26,0.115385
4501,4,4. change the line type of the grid lines,NaN,BM25,plot,5,0,0.000000,26,0.153846
4502,4,4. change the line type of the grid lines,NaN,BM25,xlabel,6,0,0.000000,26,0.192308
4503,4,4. change the line type of the grid lines,NaN,BM25,ylabel,7,0,0.000000,26,0.230769
4504,4,4. change the line type of the grid lines,NaN,BM25,yticks,8,0,0.000000,26,0.269231
4505,4,4. change the line type of the grid lines,NaN,BM25,xticks,9,0,0.000000,26,0.307692
4506,4,4. change the line type of the grid lines,NaN,BM25,legend,10,0,0.000000,26,0.346154


In [132]:
df[df.func.isnull() 
   & (df.baseline=='vectors-so-text-python-5gram.bin_func_as_seq') 
   & (df['query'] == '4. change the line type of the grid lines')][['candidate','rank','score','is_gt']]

,candidate,rank,score,is_gt
4679,ylabel,1,0.692657,0
4680,xlabel,2,0.683854,0
4681,set_facecolor,3,0.682159,0
4682,set_edgecolor,4,0.682159,0
4683,set_color,5,0.668132,0
4684,set_color,6,0.668132,0
4685,get_frame,7,0.662700,0
4686,xlim,8,0.662556,0
4687,plot,9,0.642130,0
4688,plot,10,0.642130,0


In [134]:
[w2v.model.index2word[x] for x in w2v.get_word_indexes(fu['ylabel'])]

['set', 'the', 'y_axis', 'label', 'of', 'the', 'current_axis', '.']

In [135]:
[w2v.model.index2word[x] for x in w2v.get_word_indexes(fu['grid'])]

['turn', 'the', 'axes', 'grids', 'on', 'or', 'off', '.']

In [136]:
[w2v.model.index2word[x] for x in w2v.get_word_indexes(
        'change the line type of the grid lines')]

['change', 'the', 'line', 'type_of', 'the', 'grid_lines']

In [137]:
[w2v1g.model.index2word[x] for x in w2v1g.get_word_indexes(
        'change the line type of the grid lines')]

['change', 'the', 'line', 'type', 'of', 'the', 'grid', 'lines']

In [138]:
np.dot(w2v.f_vecmat[w2v.func_lookup['grid']],
       w2v.get_bow_representation('change the line type of the grid lines').T)

0.44803807

In [139]:
np.dot(w2v.f_vecmat[w2v.func_lookup['ylabel']],
       w2v.get_bow_representation('change the line type of the grid lines').T)

0.75544059

In [278]:
from collections import defaultdict
import numpy as np
from __future__ import division
def get_idf(w2vb, documents):
    """
    Get inverse document frequency based on a given vocabulary and a given
    corpus (`documents`). For words in the vocabulary that are unseen in the
    given corpus, assign an IDF of 5.
    
    w2vb: Word2vecBaseline
    documents: a list of strings.

    Returns a list of docfreqs. Indexed by word index.
    
    """
    counts = [0] * len(w2vb.model.index2word)
    num_doc = len(documents)
    for doc in documents:
        word_idxs = set(w2vb.get_word_indexes(doc))
        for wi in word_idxs:
            counts[wi] += 1
    idfs = []
    for i in xrange(len(counts)):
        if counts[i] > 0:
            idfs.append(np.log(num_doc / counts[i]))
        else:
            # tricky !!!
            idfs.append(5)
    return idfs

In [279]:
fu_idf = get_idf(w2v, fu.values())

In [280]:
fu_idf[w2v.model.vocab['the'].index]

0.41059332447508384

In [281]:
fu_idf[w2v.model.vocab['grid'].index]

4.8700952713172558

In [282]:
fu_idf[w2v.model.vocab['grids'].index]

7.0673198486534758

In [327]:
from gensim import matutils
from numpy import float32 as REAL
def get_wbow_representation(w2vb, query, idfs):
    """
    Get weighted bag-of-word representation.
    """
    idxs = w2vb.get_word_indexes(query)    
    
    # tricky !!!
    idxs = list(set(idxs))
    
    idf_weights = [idfs[x] for x in idxs]

    # tricky !!!
    idf_weights = map(lambda x: x if x > 2 else 0, idf_weights)
        
    idf_weights = np.array(idf_weights).reshape((1,-1))
    raw_vecs = w2vb.model.syn0norm[idxs]
    weighted_sum = np.dot(idf_weights, raw_vecs)
    
    weighted_average = (weighted_sum / len(idxs))[0]
    return matutils.unitvec(weighted_average).astype(REAL)

In [328]:
np.dot(get_wbow_representation(w2v1g, fu['grid'] + ' grid', fu_idf),
       get_wbow_representation(w2v1g, 'change the line type of the grids lines', fu_idf).T)

0.35707831

In [329]:
np.dot(get_wbow_representation(w2v1g, fu['ylabel'] + 'ylabel', fu_idf),
       get_wbow_representation(w2v1g, 'change the line type of the grid lines', fu_idf).T)

0.30755258

In [330]:
np.dot(get_wbow_representation(w2v1g, fu['bar'] + ' bar', fu_idf),
       get_wbow_representation(w2v1g, 'set the color of the bar', fu_idf).T)

0.71447825

In [331]:
np.dot(get_wbow_representation(w2v1g, fu['xlim'] + ' xlim', fu_idf),
       get_wbow_representation(w2v1g, 'set the color of the bar', fu_idf).T)

0.27947184

In [332]:
np.dot(get_wbow_representation(w2v1g, fu['xlim'] + ' xlim', fu_idf),
       get_wbow_representation(w2v1g, 'set limit of the x axis', fu_idf).T)

0.68353224

In [333]:
def rank_funcs(self, query, funcs, idfs, parent=None):
    query = query.lower()
    self.model.init_sims()
    q_vec = get_wbow_representation(self, query, idfs)
    func_vecmat = np.zeros((len(funcs), self.model.vector_size))
    for i, func in enumerate(funcs):
      if self.bow:
        if func in self.func_lookup:
            func_idx = self.func_lookup[func]
            # func_vecmat[i] += self.f_vecmat[func_idx]
            # just a hack for now !!
            func_vecmat[i] += get_wbow_representation(self, fu[func] + ' ' + func, idfs)            
      else:
        if func in self.model.vocab:
          func_idx = self.model.vocab[func].index
          func_vecmat[i] += self.model.syn0norm[func_idx]
    scores = np.dot(q_vec, func_vecmat.T)
    sorted_funcs = sorted(zip(funcs, scores), key=lambda x:x[1], reverse=True)
    return sorted_funcs

In [335]:
rank_funcs(w2v1g, 'set the color of the bar', funcs, fu_idf)

[('bar', 0.71447826364391664),
 ('get_cmap', 0.39936678608537962),
 ('set_hatch', 0.30898994330988427),
 ('set_width', 0.30643947060577925),
 ('xlim', 0.2794718537902971),
 ('set_label', 0.25195645835866876),
 ('xticks', 0.24134524387616996),
 ('legend', 0.19034130906618946),
 ('title', 0.14762024748808866)]

In [336]:
funcs2 = \
df[df.func.isnull() 
   & (df.baseline=='vectors-so-text-python-5gram.bin_func_as_seq') 
   & (df['query'] == '4. change the line type of the grid lines')].candidate.tolist()
funcs2

['ylabel',
 'xlabel',
 'set_facecolor',
 'set_edgecolor',
 'set_color',
 'set_color',
 'get_frame',
 'xlim',
 'plot',
 'plot',
 'tick_params',
 'ylim',
 'yticks',
 'xticks',
 'set_visible',
 'set_visible',
 'set_linewidth',
 'set_linewidth',
 'legend',
 'grid',
 'grid',
 'get_major_ticks',
 'set_tick_params',
 'figure',
 'tick_bottom',
 'tick_left']

In [342]:
rank_funcs(w2v1g, 'change the grids', funcs2, fu_idf)

[('grid', 0.57137202715809043),
 ('grid', 0.57137202715809043),
 ('tick_params', 0.47393493532999931),
 ('plot', 0.35056293127860066),
 ('plot', 0.35056293127860066),
 ('yticks', 0.34948199292200871),
 ('ylabel', 0.32424611090492689),
 ('xlim', 0.31669246779172988),
 ('set_tick_params', 0.31562792870824641),
 ('set_color', 0.31159913345524942),
 ('set_color', 0.31159913345524942),
 ('xticks', 0.31135721308500797),
 ('xlabel', 0.30599665067096427),
 ('legend', 0.27253563456304458),
 ('tick_left', 0.26955031641372623),
 ('tick_bottom', 0.26424612065660863),
 ('ylim', 0.26356709270313328),
 ('get_frame', 0.1709388094644303),
 ('figure', 0.14614811842902958),
 ('get_major_ticks', 0.084275152931388694),
 ('set_visible', 0.017256266856496727),
 ('set_visible', 0.017256266856496727),
 ('set_facecolor', -0.047732372848420815),
 ('set_edgecolor', -0.047732372848420815),
 ('set_linewidth', -0.068463525759748109),
 ('set_linewidth', -0.068463525759748109)]